In [ ]:
import socket
import json
import pickle
import pandas as pd
import subprocess
import time


SERVER_IP = "169.254.145.243"   
PORT = 5000
ALERT_THRESHOLD = 0.7
LEAD_TIME_HOURS = 6

with open("landslide_model.pkl", "rb") as f:
    model = pickle.load(f)


sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)


last_values = {}

def sensor_health(grid_id, key, value, min_v, max_v):
    if value is None or value == 0:
        return False, "dead_sensor"

    if value < min_v or value > max_v:
        return False, "out_of_range"

    prev = last_values.get((grid_id, key))
    if prev is not None and abs(value - prev) > 0.5 * max_v:
        return False, "sudden_spike"

    last_values[(grid_id, key)] = value
    return True, "ok"

def predict_landslide(g):
    soil = g["soil_type"].lower()
    soil_gravel = 1 if soil == "gravel" else 0
    soil_sand   = 1 if soil == "sand" else 0
    soil_silt   = 1 if soil == "silt" else 0

    soil_risk = 0.2*soil_gravel + 0.6*soil_sand + 1.0*soil_silt

    df = pd.DataFrame([{
        "Rainfall_mm": g["rainfall_mm"],
        "Slope_Angle": g["slope_angle"],
        "Soil_Saturation": g["soil_saturation"],
        "Vegetation_Cover": g["vegetation_cover"],
        "Earthquake_Activity": g["earthquake_activity"],
        "Proximity_to_Water": g["proximity_to_water"],
        "Soil_Type_Gravel": soil_gravel,
        "Soil_Type_Sand": soil_sand,
        "Soil_Type_Silt": soil_silt,
        "Soil_Risk_Index": soil_risk,
        "Rain_Sand": g["rainfall_mm"] * soil_sand,
        "Slope_Silt": g["slope_angle"] * soil_silt
    }])

    prob = model.predict_proba(df)[0][1]

    if prob < 0.4:
        return prob, "SAFE"
    elif prob < ALERT_THRESHOLD:
        return prob, "MODERATE"
    else:
        return prob, "HIGH"

sensor_grids = {
    
    "grid_01": {"lat":31.101,"lon":77.170,"rainfall_mm":420,"slope_angle":55,"soil_saturation":48,"vegetation_cover":0.08,"earthquake_activity":0,"proximity_to_water":0.12,"soil_type":"silt"},
    "grid_02": {"lat":31.102,"lon":77.171,"rainfall_mm":430,"slope_angle":56,"soil_saturation":49,"vegetation_cover":0.07,"earthquake_activity":0,"proximity_to_water":0.10,"soil_type":"silt"},
    "grid_03": {"lat":31.183,"lon":77.172,"rainfall_mm":410,"slope_angle":54,"soil_saturation":47,"vegetation_cover":0.06,"earthquake_activity":0,"proximity_to_water":0.11,"soil_type":"silt"},

    
    "grid_04": {"lat":31.110,"lon":77.180,"rainfall_mm":210,"slope_angle":30,"soil_saturation":32,"vegetation_cover":0.35,"earthquake_activity":0,"proximity_to_water":0.40,"soil_type":"sand"},
    "grid_05": {"lat":31.189,"lon":77.181,"rainfall_mm":220,"slope_angle":28,"soil_saturation":30,"vegetation_cover":0.38,"earthquake_activity":0,"proximity_to_water":0.42,"soil_type":"sand"},
    "grid_06": {"lat":31.112,"lon":77.182,"rainfall_mm":200,"slope_angle":29,"soil_saturation":31,"vegetation_cover":0.40,"earthquake_activity":0,"proximity_to_water":0.41,"soil_type":"sand"},

  
    "grid_07": {"lat":31.120,"lon":77.190,"rainfall_mm":60,"slope_angle":10,"soil_saturation":12,"vegetation_cover":0.75,"earthquake_activity":0,"proximity_to_water":0.70,"soil_type":"gravel"},
    "grid_08": {"lat":31.121,"lon":77.191,"rainfall_mm":55,"slope_angle":9,"soil_saturation":10,"vegetation_cover":0.78,"earthquake_activity":0,"proximity_to_water":0.72,"soil_type":"gravel"},
    "grid_09": {"lat":31.122,"lon":77.192,"rainfall_mm":50,"slope_angle":8,"soil_saturation":9,"vegetation_cover":0.80,"earthquake_activity":0,"proximity_to_water":0.75,"soil_type":"gravel"},


    "grid_10": {"lat":31.130,"lon":79.200,"rainfall_mm":0,"slope_angle":58,"soil_saturation":0,"vegetation_cover":0.05,"earthquake_activity":0,"proximity_to_water":0.15,"soil_type":"silt"},
    "grid_11": {"lat":31.131,"lon":79.201,"rainfall_mm":480,"slope_angle":60,"soil_saturation":50,"vegetation_cover":0.04,"earthquake_activity":0,"proximity_to_water":0.12,"soil_type":"silt"},
    "grid_12": {"lat":31.132,"lon":79.202,"rainfall_mm":470,"slope_angle":59,"soil_saturation":49,"vegetation_cover":0.06,"earthquake_activity":0,"proximity_to_water":0.14,"soil_type":"silt"},

   
    "grid_13": {"lat":31.840,"lon":77.210,"rainfall_mm":0,"slope_angle":25,"soil_saturation":0,"vegetation_cover":0.45,"earthquake_activity":0,"proximity_to_water":0.50,"soil_type":"sand"},
    "grid_14": {"lat":31.941,"lon":77.211,"rainfall_mm":180,"slope_angle":26,"soil_saturation":28,"vegetation_cover":0.48,"earthquake_activity":0,"proximity_to_water":0.52,"soil_type":"sand"},
    "grid_15": {"lat":31.642,"lon":77.212,"rainfall_mm":175,"slope_angle":24,"soil_saturation":26,"vegetation_cover":0.50,"earthquake_activity":0,"proximity_to_water":0.49,"soil_type":"sand"},

  
   "grid_16": {
  "lat": 31.350,
  "lon": 77.220,
  "rainfall_mm": 15,
  "slope_angle": 3,
  "soil_saturation": 4,
  "vegetation_cover": 0.97,
  "earthquake_activity": 0,
  "proximity_to_water": 0.95,
  "soil_type": "gravel"
},

"grid_17": {
  "lat": 31.351,
  "lon": 77.221,
  "rainfall_mm": 18,
  "slope_angle": 2,
  "soil_saturation": 3,
  "vegetation_cover": 0.98,
  "earthquake_activity": 0,
  "proximity_to_water": 0.96,
  "soil_type": "gravel"
},

"grid_18": {
  "lat": 31.352,
  "lon": 77.222,
  "rainfall_mm": 20,
  "slope_angle": 4,
  "soil_saturation": 5,
  "vegetation_cover": 0.96,
  "earthquake_activity": 0,
  "proximity_to_water": 0.94,
  "soil_type": "gravel"
}
    ,
    # ZONE G
   "grid_19": {
    "lat": 33.350,
    "lon": 87.220,
    "rainfall_mm": 30,
    "slope_angle": 5,
    "soil_saturation": 8,
    "vegetation_cover": 0.95,
    "earthquake_activity": 0,
    "proximity_to_water": 0.90,
    "soil_type": "gravel"
},

"grid_20": {
    "lat": 33.351,
    "lon": 87.221,
    "rainfall_mm": 45,
    "slope_angle": 7,
    "soil_saturation": 10,
    "vegetation_cover": 0.90,
    "earthquake_activity": 0,
    "proximity_to_water": 0.85,
    "soil_type": "gravel"
},

"grid_21": {
    "lat": 33.352,
    "lon": 87.222,
    "rainfall_mm": 60,
    "slope_angle": 10,
    "soil_saturation": 12,
    "vegetation_cover": 0.88,
    "earthquake_activity": 0,
    "proximity_to_water": 0.80,
    "soil_type": "sand"
}

}


zones = {
    "ZONE_A": ["grid_01","grid_02","grid_03"],  
    "ZONE_B": ["grid_04","grid_05","grid_06"],   
    "ZONE_C": ["grid_07","grid_08","grid_09"],   
    "ZONE_D": ["grid_10","grid_11","grid_12"], 
    "ZONE_E": ["grid_13","grid_14","grid_15"],  
    "ZONE_F": ["grid_16","grid_17","grid_18"],  
    "ZONE_G": ["grid_19","grid_20","grid_21"]
  
}


grid_results = {}
faulty_detected = False
faulty_detected_grid=""

for gid, g in sensor_grids.items():
    for key, (mn, mx) in {
        "rainfall_mm":(0,600),"slope_angle":(0,60),
        "soil_saturation":(0,50),"vegetation_cover":(0,1),
        "earthquake_activity":(0,10),"proximity_to_water":(0,1)
    }.items():
        ok, _ = sensor_health(gid, key, g[key], mn, mx)
        if not ok:
            faulty_detected = True
            faulty_detected_grid=gid
            g[key] = (mn + mx) / 2

    prob, risk = predict_landslide(g)
    grid_results[gid] = {"prob":prob,"risk":risk}

def aggregate_zone(zone_id, grids):
    lats, lons, probs = [], [], []
    votes = {"SAFE":0,"MODERATE":0,"HIGH":0}

    for gid in grids:
        lats.append(sensor_grids[gid]["lat"])
        lons.append(sensor_grids[gid]["lon"])
        probs.append(grid_results[gid]["prob"])
        votes[grid_results[gid]["risk"]] += 1

    return {
        "zone_id": zone_id,
        "zone_size_m": 100,
        "latitude": round(sum(lats)/len(lats),6),
        "longitude": round(sum(lons)/len(lons),6),
        "risk": max(votes, key=votes.get),
        "probability": round(sum(probs)/len(probs),2),
        "lead_time_hours": LEAD_TIME_HOURS
    }


zone_alert = False

for zid, glist in zones.items():
    payload = aggregate_zone(zid, glist)
    sock.sendto(json.dumps(payload).encode(), (SERVER_IP, PORT))
    print("Dashboard:", payload)

    if payload["risk"] in ["MODERATE","HIGH"]:
        zone_alert = True



if zone_alert:
    sms = "LANDSLIDE WARNING: High risk in your area within next 6 hours. Move to a safe location immediately."
    subprocess.run(f'adb shell "echo \'{sms}\' > /storage/emulated/0/Download/sensor.txt"', shell=True)
time.sleep(2.5)

if faulty_detected: 
    sms = f"SENSOR ALERT: Some ground sensors malfunctioning. System running in degraded but safe mode. faulty grid {faulty_detected_grid}"
    subprocess.run(f'adb shell "echo \'{sms}\' > /storage/emulated/0/Download/sensor.txt"', shell=True)


c:\Users\HP\.conda\envs\chatbot\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\HP\.conda\envs\chatbot\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Dashboard: {'zone_id': 'ZONE_A', 'zone_size_m': 100, 'latitude': 31.128667, 'longitude': 77.171, 'risk': 'HIGH', 'probability': np.float64(0.84), 'lead_time_hours': 6}
Dashboard: {'zone_id': 'ZONE_B', 'zone_size_m': 100, 'latitude': 31.137, 'longitude': 77.181, 'risk': 'HIGH', 'probability': np.float64(0.93), 'lead_time_hours': 6}
Dashboard: {'zone_id': 'ZONE_C', 'zone_size_m': 100, 'latitude': 31.121, 'longitude': 77.191, 'risk': 'MODERATE', 'probability': np.float64(0.58), 'lead_time_hours': 6}
Dashboard: {'zone_id': 'ZONE_D', 'zone_size_m': 100, 'latitude': 31.131, 'longitude': 79.201, 'risk': 'HIGH', 'probability': np.float64(0.88), 'lead_time_hours': 6}
Dashboard: {'zone_id': 'ZONE_E', 'zone_size_m': 100, 'latitude': 31.807667, 'longitude': 77.211, 'risk': 'HIGH', 'probability': np.float64(0.8), 'lead_time_hours': 6}
Dashboard: {'zone_id': 'ZONE_F', 'zone_size_m': 100, 'latitude': 31.351, 'longitude': 77.221, 'risk': 'MODERATE', 'probability': np.float64(0.58), 'lead_time_hours': 